### Deliverable 3: Optimize the Model

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf


In [ ]:
from pathlib import Path
print(Path.cwd())

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Starter_code/charity_data.csv")
application_df.head()

### Preprocessing data

In [3]:
# Drop the non-beneficial ID columns, 'EIN', 'NAME', "STATUS" and "SPECIAL_CONSIDERATIONS".
application_df.drop(["EIN", "NAME", "STATUS", "SPECIAL_CONSIDERATIONS"], axis=1,inplace=True)

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_counts = application_df["APPLICATION_TYPE"].value_counts()

# Determine which values to replace if counts are less than ...?
replace_application = list(application_counts[application_counts<150].index)

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Look at CLASSIFICATION value counts for binning
classification_counts = application_df["CLASSIFICATION"].value_counts()

# Determine which values to replace if counts are less than ..?
replace_classification = list(classification_counts[classification_counts<1000].index)

# Replace in dataframe
for cls in replace_classification:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
#application_df.CLASSIFICATION.value_counts()

In [6]:
# Look at ASK_AMT value counts for binning
ask_counts = application_df["ASK_AMT"].value_counts()

# Determine which values to replace if counts are less than ..?
replace_ask = list(ask_counts[ask_counts<5000].index)

# Replace in dataframe
for ak in replace_ask:
    application_df.ASK_AMT = application_df.ASK_AMT.replace(ak,"Other")
    
# Check to make sure binning was successful
#application_df.ASK_AMT.value_counts()

In [7]:
# Look at INCOME_AMT value counts for binning
#income_counts = application_df["INCOME_AMT"].value_counts()

# Determine which values to replace if counts are less than ..?
#replace_income = list(income_counts[income_counts<1000].index)

# Replace in dataframe
#for inc in replace_income:
#    application_df.INCOME_AMT = application_df.INCOME_AMT.replace(inc,"Other")
    
# Check to make sure binning was successful
#application_df.INCOME_AMT.value_counts()

In [8]:
# Look at AFFILIATION value counts for binning
#affiliation_counts = application_df["AFFILIATION"].value_counts()

# Determine which values to replace if counts are less than ..?
#replace_affiliation = list(affiliation_counts[affiliation_counts<1000].index)

# Replace in dataframe
#for afl in replace_affiliation:
#    application_df.AFFILIATION = application_df.AFFILIATION.replace(afl,"Other")
    
# Check to make sure binning was successful
#application_df.AFFILIATION.value_counts()

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == 'object'].index.tolist()

In [10]:
#Convert datatypes - Encoders require their input to be uniformly
application_df = (application_df).astype(str)

In [11]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))
# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)


In [12]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df,left_index=True,right_index=True)
application_df = application_df.drop(application_cat,axis=1)
#application_df.head()

In [13]:
# Convert the "IS_SUCCESSFUL" column back to int for the x train model to work
application_df['IS_SUCCESSFUL'] = pd.to_numeric(application_df['IS_SUCCESSFUL'])
#application_df["IS_SUCCESSFUL"].dtypes

In [14]:
# Split our preprocessed data into our features and target arrays
X=application_df.drop(["IS_SUCCESSFUL"],axis=1)
y=application_df[["IS_SUCCESSFUL"]]
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [15]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Design a neural network model

In [16]:

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
num_input_features = len(X_train_scaled[0])
nodes_hidden_layer1 =100
nodes_hidden_layer2 = 30
#nodes_hidden_layer3 = 10
#nodes_hidden_layer4 = 50
#nodes_hidden_layer5 = 25

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer1, activation="relu", input_dim=num_input_features))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer2, activation="sigmoid"))

# Third hidden layer
#nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer3, activation="sigmoid"))

# Fourth hidden layer
#nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer4, activation="sigmoid"))

# Fifth hidden layer
#nn.add(tf.keras.layers.Dense(units=nodes_hidden_layer5, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               4300      
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 7,361
Trainable params: 7,361
Non-trainable params: 0
_________________________________________________________________


2023-01-22 22:21:25.511428: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
# Checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Checkpoint path and file name
os.makedirs("Assignment_checkpoint/",exist_ok=True)
checkpoint_path = "Assignment_checkpoint/weights.{epoch:o2d}.hd5c" 

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [28]:
# Callback that saves the model's weight every 5 epochs
cp_callback = ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5)

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)#,callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 1s 664us/step - loss: 0.5301 - accuracy: 0.7428
Epoch 2/100
804/804 [==============================] - 1s 657us/step - loss: 0.5298 - accuracy: 0.7426
Epoch 3/100
804/804 [==============================] - 1s 663us/step - loss: 0.5295 - accuracy: 0.7428
Epoch 4/100
804/804 [==============================] - 1s 658us/step - loss: 0.5297 - accuracy: 0.7425
Epoch 5/100
804/804 [==============================] - 1s 660us/step - loss: 0.5295 - accuracy: 0.7429
Epoch 6/100
804/804 [==============================] - 1s 666us/step - loss: 0.5294 - accuracy: 0.7429
Epoch 7/100
804/804 [==============================] - 1s 655us/step - loss: 0.5304 - accuracy: 0.7427
Epoch 8/100
804/804 [==============================] - 1s 660us/step - loss: 0.5300 - accuracy: 0.7421
Epoch 9/100
804/804 [==============================] - 1s 660us/step - loss: 0.5294 - accuracy: 0.7428
Epoch 10/100
804/804 [==============================] - 1s 671us/step - l

804/804 [==============================] - 1s 654us/step - loss: 0.5281 - accuracy: 0.7434
Epoch 80/100
804/804 [==============================] - 1s 656us/step - loss: 0.5285 - accuracy: 0.7432
Epoch 81/100
804/804 [==============================] - 1s 675us/step - loss: 0.5288 - accuracy: 0.7425
Epoch 82/100
804/804 [==============================] - 1s 680us/step - loss: 0.5282 - accuracy: 0.7434
Epoch 83/100
804/804 [==============================] - 1s 652us/step - loss: 0.5275 - accuracy: 0.7434
Epoch 84/100
804/804 [==============================] - 1s 655us/step - loss: 0.5279 - accuracy: 0.7425
Epoch 85/100
804/804 [==============================] - 1s 657us/step - loss: 0.5279 - accuracy: 0.7431
Epoch 86/100
804/804 [==============================] - 1s 659us/step - loss: 0.5273 - accuracy: 0.7439
Epoch 87/100
804/804 [==============================] - 1s 650us/step - loss: 0.5283 - accuracy: 0.7432
Epoch 88/100
804/804 [==============================] - 1s 661us/step - loss:

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5646 - accuracy: 0.7252 - 144ms/epoch - 537us/step
Loss: 0.5645833015441895, Accuracy: 0.7252478003501892


In [43]:
nn.save("AlphabetSoupCharity.h5c")

INFO:tensorflow:Assets written to: AlphabetSoupCharity.h5c/assets


INFO:tensorflow:Assets written to: AlphabetSoupCharity.h5c/assets
